# Preprocessing CGE data

In [1]:
# the following code lines are necessary to import custom module
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import boto3
import io
import json
import pandas as pd
from src.preprocessing.data_preparation_and_cleaning import get_station_name
from src.preprocessing.feature_engineer import get_wind_components, resample_data, feature_engineer
from unidecode import unidecode

In [2]:
# AWS credentials to read files on S3 bucket
f = open('../credentials.json')
credentials = json.load(f)

s3_client = boto3.client(
    "s3",
    aws_access_key_id=credentials["Access key ID"],
    aws_secret_access_key=credentials["Secret access key"]
    )

s3_resource = boto3.resource(
    "s3",
    aws_access_key_id=credentials["Access key ID"],
    aws_secret_access_key=credentials["Secret access key"]
    )

In [3]:
# get data from S3 bucket
df_full = []
for year in list(range(2009, 2019+1)):  
    prefix_objs = s3_resource.Bucket("cge").objects.filter(Prefix=f"raw/{year}/")
    keys = [obj.key for obj in prefix_objs]
    for key in keys:
        obj = s3_client.get_object(Bucket="cge", Key=key)
        df = pd.read_csv(io.BytesIO(obj["Body"].read()))
        df_full.append(df.rename(columns={"PressÃ£o(mb)": "Pressão(mb)"}))

In [4]:
# concatenate data and selecting stations to work
cge = pd.concat(df_full, ignore_index=True)
cge = cge.drop_duplicates(ignore_index=True)
cge = get_station_name(cge)
cge[["Posto Nome"]] = cge[["Posto Nome"]].replace("NaN", "1000300")
stations = list(cge["Posto Nome"].unique())
stations.remove("1000300")
cge = cge[cge["Posto Nome"].isin(stations)]
cge = cge.drop(["Bateria(V)", "Sens. Térmica(°C)"], axis=1)
cge.head()

,Posto,DATA,PLU(mm),Vel.VT(m/s),Dir.VT(o),Temp(oC),Umid.Rel.(%),Pressão(mb),Rajada.VT(m/s),Posto Nome
0,1000842,2009-01-01 00:00:00,0.0,NaN,NaN,NaN,NaN,NaN,NaN,Butantã
1,1000842,2009-01-01 00:10:00,0.0,NaN,NaN,NaN,NaN,NaN,NaN,Butantã
2,1000842,2009-01-01 00:20:00,0.0,NaN,NaN,NaN,NaN,NaN,NaN,Butantã
3,1000842,2009-01-01 00:30:00,0.0,NaN,NaN,NaN,NaN,NaN,NaN,Butantã
4,1000842,2009-01-01 00:40:00,0.0,NaN,NaN,NaN,NaN,NaN,NaN,Butantã


In [5]:
obj = s3_client.get_object(Bucket="cge", Key="processed/cge_clusters.csv")
clusters = pd.read_csv(io.BytesIO(obj["Body"].read()))
clusters = clusters[["station", "cluster"]].rename(columns={"station": "Posto Nome"})
clusters

,Posto Nome,cluster
0,Anhembi,A
1,Butantã,A
2,Campo Limpo,B
3,Capela do Socorro,D
4,Cidade Ademar,B
5,Freguesia do Ó,A
6,Ipiranga,B
7,Itaim Paulista,B
8,Itaquera,B
9,Jabaquara,A


In [6]:
cge = cge.merge(
    clusters,
    on="Posto Nome",
    how="left",
)

cge

,Posto,DATA,PLU(mm),Vel.VT(m/s),Dir.VT(o),Temp(oC),Umid.Rel.(%),Pressão(mb),Rajada.VT(m/s),Posto Nome,cluster
0,1000842,2009-01-01 00:00:00,0.0,NaN,NaN,NaN,NaN,NaN,NaN,Butantã,A
1,1000842,2009-01-01 00:10:00,0.0,NaN,NaN,NaN,NaN,NaN,NaN,Butantã,A
2,1000842,2009-01-01 00:20:00,0.0,NaN,NaN,NaN,NaN,NaN,NaN,Butantã,A
3,1000842,2009-01-01 00:30:00,0.0,NaN,NaN,NaN,NaN,NaN,NaN,Butantã,A
4,1000842,2009-01-01 00:40:00,0.0,NaN,NaN,NaN,NaN,NaN,NaN,Butantã,A
...,...,...,...,...,...,...,...,...,...,...,...
11732659,635,2019-02-18 22:10:00,0.0,NaN,NaN,24.346,84.251,926.89,NaN,Pinheiros,A
11732660,635,2019-02-18 22:20:00,0.0,NaN,NaN,24.175,85.307,927.39,NaN,Pinheiros,A
11732661,635,2019-02-18 22:30:00,0.0,NaN,NaN,24.144,86.040,927.48,NaN,Pinheiros,A
11732662,635,2019-02-18 22:40:00,0.0,NaN,NaN,24.104,86.168,927.71,NaN,Pinheiros,A


In [7]:
# renaming columns
cge.columns = [
    "station",
    "timestamp",
    "precipitation",
    "wind_velocity",
    "wind_direction",
    "temperature",
    "relative_humidity",
    "pressure",
    "wind_blow",
    "station_name",
    "cluster"
]

cge.head()

,station,timestamp,precipitation,wind_velocity,wind_direction,temperature,relative_humidity,pressure,wind_blow,station_name,cluster
0,1000842,2009-01-01 00:00:00,0.0,NaN,NaN,NaN,NaN,NaN,NaN,Butantã,A
1,1000842,2009-01-01 00:10:00,0.0,NaN,NaN,NaN,NaN,NaN,NaN,Butantã,A
2,1000842,2009-01-01 00:20:00,0.0,NaN,NaN,NaN,NaN,NaN,NaN,Butantã,A
3,1000842,2009-01-01 00:30:00,0.0,NaN,NaN,NaN,NaN,NaN,NaN,Butantã,A
4,1000842,2009-01-01 00:40:00,0.0,NaN,NaN,NaN,NaN,NaN,NaN,Butantã,A


In [8]:
cluster_A = []
cluster_B = []
cluster_C = []
cluster_D = []

# feature engineer and write data on S3 bucket
for station in stations:
    df = cge[cge.station_name == station]
    cluster = df.cluster.unique()[0]
    df = df.dropna(axis=1, how="all")
    df = df.reset_index(drop=True) 

    try:
        df = get_wind_components(
            df=df, 
            wind_velocity="wind_velocity", 
            wind_direction="wind_direction", 
            x_name="wind_velocity_x", 
            y_name="wind_velocity_y"
            )

        df = get_wind_components(
            df=df, 
            wind_velocity="wind_blow", 
            wind_direction="wind_direction", 
            x_name="wind_blow_x", 
            y_name="wind_blow_y"
            )

        df = df.drop(["wind_velocity", "wind_blow", "wind_direction"], axis=1)
    except:
        pass

    df = resample_data(df)
    df = df.reset_index()
    df = df.set_index("timestamp")
    to_drop = ["station", "station_name"]
    cols = list(df.drop(to_drop, axis=1).columns)
    df = feature_engineer(df=df, features=cols, lags=12, window=12)
    cols.remove("temperature")
    df = df.drop(cols, axis=1)
    df["cluster"] = cluster

    if cluster == "A":
        cluster_A.append(df)
    elif cluster == "B":
        cluster_B.append(df)
    elif cluster == "C":
        cluster_C.append(df)
    elif cluster == "D":
        cluster_D.append(df)


def writing_data_to_S3(df, file_name):
    buffer = io.StringIO()
    df.to_csv(buffer)
    s3_resource.Object("cge", f"processed/{file_name}.csv").put(Body=buffer.getvalue())

/home/ubuntu/ml-uhi/src/preprocessing/feature_engineer.py:29: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.resample(rule="60min").mean()
/home/ubuntu/ml-uhi/src/preprocessing/feature_engineer.py:29: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.resample(rule="60min").mean()
/home/ubuntu/ml-uhi/src/preprocessing/feature_engineer.py:29: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.resample(rule

In [9]:
cluster_A = pd.concat(cluster_A).groupby(["timestamp", "cluster"]).mean().reset_index()
writing_data_to_S3(df=cluster_A, file_name="cluster_A")

/tmp/ipykernel_1730/2380661146.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  cluster_A = pd.concat(cluster_A).groupby(["timestamp", "cluster"]).mean().reset_index()


In [10]:
cluster_A

,timestamp,cluster,temperature,station,month_sin,month_cos,hour_sin,hour_cos,precipitation_lag_1H,temperature_lag_1H,...,wind_velocity_y_window_12H_min,wind_velocity_y_window_12H_max,wind_blow_x_window_12H_mean,wind_blow_x_window_12H_std,wind_blow_x_window_12H_min,wind_blow_x_window_12H_max,wind_blow_y_window_12H_mean,wind_blow_y_window_12H_std,wind_blow_y_window_12H_min,wind_blow_y_window_12H_max
0,2009-01-01 00:00:00,A,NaN,1.000856e+06,0.500000,0.866025,0.000000e+00,1.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2009-01-01 01:00:00,A,NaN,1.000856e+06,0.500000,0.866025,2.697968e-01,0.962917,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2009-01-01 02:00:00,A,NaN,1.000856e+06,0.500000,0.866025,5.195840e-01,0.854419,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2009-01-01 03:00:00,A,NaN,1.000856e+06,0.500000,0.866025,7.308360e-01,0.682553,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2009-01-01 04:00:00,A,NaN,1.000856e+06,0.500000,0.866025,8.878852e-01,0.460065,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88819,2019-02-18 19:00:00,A,28.352578,4.673621e+05,0.866025,0.500000,-8.878852e-01,0.460065,0.0,28.953400,...,-0.752864,0.817271,1.136658,1.425815,-0.935801,2.941374,0.148269,1.179696,-1.945238,1.753045
88820,2019-02-18 20:00:00,A,26.339022,4.673621e+05,0.866025,0.500000,-7.308360e-01,0.682553,0.0,28.352578,...,-1.013479,0.817271,1.238500,1.341978,-0.891170,2.965928,0.029326,1.310700,-2.418097,1.776884
88821,2019-02-18 21:00:00,A,24.735967,4.673621e+05,0.866025,0.500000,-5.195840e-01,0.854419,0.0,26.339022,...,-1.161314,0.821876,1.295706,1.431589,-1.164759,3.379662,-0.088646,1.441772,-2.748483,1.854892
88822,2019-02-18 22:00:00,A,23.812019,4.292491e+05,0.866025,0.500000,-2.697968e-01,0.962917,0.0,24.677155,...,-1.290846,0.978555,1.112079,1.713146,-2.056593,3.367360,-0.241454,1.616738,-3.059747,2.099696


In [11]:
cluster_B = pd.concat(cluster_B).groupby(["timestamp", "cluster"]).mean().reset_index()
writing_data_to_S3(df=cluster_B, file_name="cluster_B")

/tmp/ipykernel_1730/4193850894.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  cluster_B = pd.concat(cluster_B).groupby(["timestamp", "cluster"]).mean().reset_index()


In [12]:
cluster_B

,timestamp,cluster,temperature,station,month_sin,month_cos,hour_sin,hour_cos,precipitation_lag_1H,temperature_lag_1H,...,wind_direction_window_11H_min,wind_direction_window_11H_max,wind_velocity_window_12H_mean,wind_velocity_window_12H_std,wind_velocity_window_12H_min,wind_velocity_window_12H_max,wind_direction_window_12H_mean,wind_direction_window_12H_std,wind_direction_window_12H_min,wind_direction_window_12H_max
0,2009-09-14 10:00:00,B,NaN,1000880.0,-1.000000,-1.836970e-16,0.398401,-0.917211,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2009-09-14 11:00:00,B,NaN,1000880.0,-1.000000,-1.836970e-16,0.136167,-0.990686,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2009-09-14 12:00:00,B,NaN,1000880.0,-1.000000,-1.836970e-16,-0.136167,-0.990686,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2009-09-14 13:00:00,B,NaN,1000880.0,-1.000000,-1.836970e-16,-0.398401,-0.917211,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2009-09-14 14:00:00,B,NaN,1000880.0,-1.000000,-1.836970e-16,-0.631088,-0.775711,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82664,2019-02-18 18:00:00,B,27.692857,625744.5,0.866025,5.000000e-01,-0.979084,0.203456,0.0,28.055952,...,109.057667,130.272833,0.365556,0.382200,0.000000,1.128000,120.617736,6.533956,109.057667,130.272833
82665,2019-02-18 19:00:00,B,26.650024,625744.5,0.866025,5.000000e-01,-0.887885,0.460065,0.0,27.692857,...,109.057667,130.272833,0.423000,0.373977,0.000000,1.128000,121.508875,6.535770,109.057667,130.272833
82666,2019-02-18 20:00:00,B,24.810286,625744.5,0.866025,5.000000e-01,-0.730836,0.682553,0.0,26.650024,...,109.057667,130.272833,0.483056,0.357369,0.000000,1.128000,121.563181,6.498213,109.057667,130.272833
82667,2019-02-18 21:00:00,B,23.552119,625744.5,0.866025,5.000000e-01,-0.519584,0.854419,0.0,24.810286,...,106.188833,130.272833,0.545722,0.329835,0.000000,1.128000,120.392292,7.877823,106.188833,130.272833


In [13]:
cluster_C = pd.concat(cluster_C).groupby(["timestamp", "cluster"]).mean().reset_index()
writing_data_to_S3(df=cluster_C, file_name="cluster_C")

/tmp/ipykernel_1730/1045015748.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  cluster_C = pd.concat(cluster_C).groupby(["timestamp", "cluster"]).mean().reset_index()


In [14]:
cluster_C

,timestamp,cluster,temperature,station,month_sin,month_cos,hour_sin,hour_cos,precipitation_lag_1H,temperature_lag_1H,...,wind_velocity_y_window_12H_min,wind_velocity_y_window_12H_max,wind_blow_x_window_12H_mean,wind_blow_x_window_12H_std,wind_blow_x_window_12H_min,wind_blow_x_window_12H_max,wind_blow_y_window_12H_mean,wind_blow_y_window_12H_std,wind_blow_y_window_12H_min,wind_blow_y_window_12H_max
0,2009-01-01 00:00:00,C,NaN,1000856.0,0.500000,0.866025,0.000000,1.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2009-01-01 01:00:00,C,NaN,1000856.0,0.500000,0.866025,0.269797,0.962917,2.9,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2009-01-01 02:00:00,C,NaN,1000856.0,0.500000,0.866025,0.519584,0.854419,2.9,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2009-01-01 03:00:00,C,NaN,1000856.0,0.500000,0.866025,0.730836,0.682553,2.9,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2009-01-01 04:00:00,C,NaN,1000856.0,0.500000,0.866025,0.887885,0.460065,2.9,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88818,2019-02-18 18:00:00,C,28.309867,1000856.8,0.866025,0.500000,-0.979084,0.203456,0.0,29.270067,...,-0.790286,0.777811,2.114934,1.751569,-0.050643,4.756649,0.139021,1.154319,-2.107168,1.574646
88819,2019-02-18 19:00:00,C,26.799067,1000856.8,0.866025,0.500000,-0.887885,0.460065,0.0,28.309867,...,-1.341158,0.777811,2.078409,1.918911,-1.214608,4.756649,-0.121644,1.564739,-3.714114,1.574646
88820,2019-02-18 20:00:00,C,24.551900,1000856.8,0.866025,0.500000,-0.730836,0.682553,0.0,26.799067,...,-1.341158,0.777811,1.822543,2.322136,-2.559895,4.756649,-0.293429,1.709882,-3.714114,1.574646
88821,2019-02-18 21:00:00,C,23.272433,1000856.8,0.866025,0.500000,-0.519584,0.854419,0.0,24.551900,...,-1.341158,0.777811,1.530332,2.715303,-3.400221,4.756649,-0.299653,1.779376,-3.714114,1.574646


In [15]:
cluster_D = pd.concat(cluster_D).groupby(["timestamp", "cluster"]).mean().reset_index()
writing_data_to_S3(df=cluster_D, file_name="cluster_D")

/tmp/ipykernel_1730/2649288607.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  cluster_D = pd.concat(cluster_D).groupby(["timestamp", "cluster"]).mean().reset_index()


In [16]:
cluster_D

,timestamp,cluster,temperature,station,month_sin,month_cos,hour_sin,hour_cos,precipitation_lag_1H,temperature_lag_1H,...,wind_velocity_y_window_12H_min,wind_velocity_y_window_12H_max,wind_blow_x_window_12H_mean,wind_blow_x_window_12H_std,wind_blow_x_window_12H_min,wind_blow_x_window_12H_max,wind_blow_y_window_12H_mean,wind_blow_y_window_12H_std,wind_blow_y_window_12H_min,wind_blow_y_window_12H_max
0,2009-01-01 00:00:00,D,NaN,1000857.0,0.500000,0.866025,0.000000,1.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2009-01-01 01:00:00,D,NaN,1000857.0,0.500000,0.866025,0.269797,0.962917,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2009-01-01 02:00:00,D,NaN,1000857.0,0.500000,0.866025,0.519584,0.854419,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2009-01-01 03:00:00,D,NaN,1000857.0,0.500000,0.866025,0.730836,0.682553,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2009-01-01 04:00:00,D,NaN,1000857.0,0.500000,0.866025,0.887885,0.460065,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88818,2019-02-18 18:00:00,D,25.671000,500628.5,0.866025,0.500000,-0.979084,0.203456,0.0,27.257167,...,-2.06188,0.820437,2.029777,1.628234,-2.348914,4.291259,-0.047099,1.615708,-2.945281,2.410996
88819,2019-02-18 19:00:00,D,24.605000,500628.5,0.866025,0.500000,-0.887885,0.460065,0.0,25.671000,...,-2.06188,0.820437,1.689979,2.049334,-2.348914,4.291259,0.006243,1.666354,-2.945281,2.410996
88820,2019-02-18 20:00:00,D,22.963500,500628.5,0.866025,0.500000,-0.730836,0.682553,0.0,24.605000,...,-2.06188,0.892353,1.339505,2.423730,-2.778191,4.291259,0.086801,1.743626,-2.945281,2.410996
88821,2019-02-18 21:00:00,D,21.979000,500628.5,0.866025,0.500000,-0.519584,0.854419,0.0,22.963500,...,-2.06188,0.892353,1.023031,2.623873,-2.778191,4.291259,0.125222,1.772499,-2.945281,2.410996
